# Chapter 10. Machine Learning with MLlib

In [ ]:
from pyspark import SparkContext
from pyspark.sql import SparkSession
import pyspark.sql.functions as F

spark = (SparkSession.builder
  .master("local[4]")
  .appName("MachineLearning")
  .getOrCreate())
spark.sparkContext.setLogLevel("ERROR")
spark

## Designing Machine Learning Pipelines

### Data Ingestion and Exploration

In [ ]:
filePath = "../data/sf-airbnb/sf-airbnb-clean.parquet/"
airbnbDF = spark.read.parquet(filePath)
airbnbDF.select("neighbourhood_cleansed", "room_type", "bedrooms", "bathrooms", 
                "number_of_reviews", "price").show(5)

### Creating Training and Test Data Sets

In [ ]:
trainDF, testDF = airbnbDF.randomSplit([.8, .2], seed=42)
print(f"""There are {trainDF.count()} rows in the training set, and {testDF.count()} in the test set""")

### Preparing Features with Transformers

In [ ]:
from pyspark.ml.feature import VectorAssembler
vecAssembler = VectorAssembler(inputCols=["bedrooms"], outputCol="features")
vecTrainDF = vecAssembler.transform(trainDF)
vecTrainDF.select("bedrooms", "features", "price").show(10)

### Using Estimators to Build Models

In [ ]:
from pyspark.ml.regression import LinearRegression
lr = LinearRegression(featuresCol="features", labelCol="price")
lrModel = lr.fit(vecTrainDF)

In [ ]:
# Inspect the learned parameters
m = round(lrModel.coefficients[0], 2)
b = round(lrModel.intercept, 2)
print(f"""The formula for the linear regression line is 
price = {m}*bedrooms + {b}""")

### Creating a Pipeline

In [ ]:
# Create a pipelin
from pyspark.ml import Pipeline
pipeline = Pipeline(stages=[vecAssembler, lr])
pipelineModel = pipeline.fit(trainDF)

In [ ]:
# Apply the model to the test data set
predDF = pipelineModel.transform(testDF)
predDF.select("bedrooms", "features", "price", "prediction").show(10)

#### One-hot encoding

In [ ]:
from pyspark.ml.feature import OneHotEncoder, StringIndexer

categoricalCols = [field for (field, dataType) in trainDF.dtypes 
                   if dataType == "string"]
indexOutputCols = [x + "Index" for x in categoricalCols]
oheOutputCols = [x + "OHE" for x in categoricalCols]

stringIndexer = StringIndexer(inputCols=categoricalCols, 
                              outputCols=indexOutputCols, 
                              handleInvalid="skip")
oheEncoder = OneHotEncoder(inputCols=indexOutputCols, 
                           outputCols=oheOutputCols)

numericCols = [field for (field, dataType) in trainDF.dtypes 
               if ((dataType == "double") & (field != "price"))]
assemblerInputs = oheOutputCols + numericCols
vecAssembler = VectorAssembler(inputCols=assemblerInputs, 
                               outputCol="features")

In [ ]:
# Doing the same as in the previous cell, but using RFormula
from pyspark.ml.feature import RFormula

rFormula = RFormula(formula="price ~ .", 
                    featuresCol="features", 
                    labelCol="price", 
                    handleInvalid="skip")

In [ ]:
# In Python
lr = LinearRegression(labelCol="price", featuresCol="features")
pipeline = Pipeline(stages = [stringIndexer, oheEncoder, vecAssembler, lr])
# Or use RFormula
# pipeline = Pipeline(stages = [rFormula, lr])

pipelineModel = pipeline.fit(trainDF)
predDF = pipelineModel.transform(testDF)
predDF.select("features", "price", "prediction").show(5, truncate=False)

### Evaluating Models

#### RMSE

In [ ]:
# In Python
from pyspark.ml.evaluation import RegressionEvaluator
regressionEvaluator = RegressionEvaluator(
  predictionCol="prediction", 
  labelCol="price", 
  metricName="rmse")
rmse = regressionEvaluator.evaluate(predDF)
print(f"RMSE is {rmse:.1f}")

##### R^2

In [ ]:
r2 = regressionEvaluator.setMetricName("r2").evaluate(predDF)
print(f"R2 is {r2}")

### Saving and Loading Models

In [ ]:
pipelinePath = "../data_output/lr-pipeline-model"
pipelineModel.write().overwrite().save(pipelinePath)

In [ ]:
from pyspark.ml import PipelineModel
savedPipelineModel = PipelineModel.load(pipelinePath)
savedPipelineModel.transform(testDF).select("features", "price", "prediction").show(5)

## Hyperparameter Tuning

### Tree-Based Models

#### Decision trees

In [ ]:
from pyspark.ml.regression import DecisionTreeRegressor

dt = DecisionTreeRegressor(labelCol="price")

# Filter for just numeric columns (and exclude price, our label)
numericCols = [field for (field, dataType) in trainDF.dtypes 
               if ((dataType == "double") & (field != "price"))]

# Combine output of StringIndexer defined above and numeric columns
assemblerInputs = indexOutputCols + numericCols
vecAssembler = VectorAssembler(inputCols=assemblerInputs, outputCol="features")

# Combine stages into pipeline
stages = [stringIndexer, vecAssembler, dt]
pipeline = Pipeline(stages=stages)
pipelineModel = pipeline.fit(trainDF) # This line should error

In [ ]:
# Increase `maxBins` to handle the discretization of the categorical columns
dt.setMaxBins(40)
pipelineModel = pipeline.fit(trainDF)

In [ ]:
# Extract the if-then-else rules learned by the decision tree
dtModel = pipelineModel.stages[-1]
print(dtModel.toDebugString)

In [ ]:
# Extract the feature importance scores from our model to see the most important features
import pandas as pd

featureImp = pd.DataFrame(
  list(zip(vecAssembler.getInputCols(), dtModel.featureImportances)),
  columns=["feature", "importance"])
featureImp.sort_values(by="importance", ascending=False)

#### Random forests

In [ ]:
from pyspark.ml.regression import RandomForestRegressor
rf = RandomForestRegressor(labelCol="price", maxBins=40, seed=42)

### k-Fold Cross-Validation

In [ ]:
# Defining pipeline estimator
pipeline = Pipeline(stages = [stringIndexer, vecAssembler, rf])

In [ ]:
# Set up our hyperparameter grid
from pyspark.ml.tuning import ParamGridBuilder
paramGrid = (ParamGridBuilder()
            .addGrid(rf.maxDepth, [2, 4, 6])
            .addGrid(rf.numTrees, [10, 100])
            .build())

In [ ]:
# Define how to evaluate each of the models to determine which one performed best
evaluator = RegressionEvaluator(labelCol="price", 
                                predictionCol="prediction", 
                                metricName="rmse")

In [ ]:
# Perform k-fold cross-validation
from pyspark.ml.tuning import CrossValidator

cv = CrossValidator(estimator=pipeline, 
                    evaluator=evaluator, 
                    estimatorParamMaps=paramGrid, 
                    numFolds=3, 
                    seed=42)

In [ ]:
%%timeit
cvModel = cv.fit(trainDF)

In [ ]:
# Inspect the results of the cross-validator
list(zip(cvModel.getEstimatorParamMaps(), cvModel.avgMetrics))

### Optimizing Pipelines

In [ ]:
%%timeit
cvModel = cv.setParallelism(10).fit(trainDF)

In [ ]:
cv = CrossValidator(estimator=rf, 
                    evaluator=evaluator, 
                    estimatorParamMaps=paramGrid, 
                    numFolds=3, 
                    parallelism=10, 
                    seed=42)

pipeline = Pipeline(stages=[stringIndexer, vecAssembler, cv])

In [ ]:
%%timeit
pipelineModel = pipeline.fit(trainDF)